<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-03-02 09:44:39
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.35 C
-------------------
Today PnL: -1.73 L (-1.26%)
Current PnL: -39.16 L (-24.48%)
CY Booked + Current PnL: -23.48 L (-14.68%)
-------------------
Total profit:  1.18 L
Total loss:  -40.34 L
-------------------
Total Booked + Current PnL: 3.50 L (2.63%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.59%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.06 C (78.19%)
Deployed:  1.33 C
Current:  1.35 C
CAGR/XIRR %: 0.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-0.79,5.07,12.00,17.67,18970.0,7622.0,158082.0,478.51,26.26,53.0,L-LC,10.87,185.0,0.40,1.19,26.90,XY25,ATH,MINING
50,MASFIN,-1.65,-2.33,24.46,21.57,23408.0,-2280.0,95700.0,397.04,-15.99,44.0,H-SC,6.75,167.0,-0.10,0.72,39.66,XR,ATH,FINANCE
77,TTKPRESTIG,-1.79,-33.27,50.00,0.09,33624.0,-33528.0,67249.0,770.00,56.03,28.0,M-SC,11.96,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-0.83,-5.70,31.57,24.08,37794.0,-7234.0,119716.0,1486.00,-6.31,47.0,X-SC,7.96,87.0,-0.19,0.90,9.65,XY24,NTT,HEALTHCARE
37,ICICIGI,-0.72,0.00,19.34,19.34,38703.0,8.0,200117.0,2252.93,-15.91,46.0,X-MC,1.16,66.0,0.00,1.51,15.94,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,2.34,-12.49,66.34,45.57,114447.0,-24616.0,172516.0,844.0,3743.24,61.0,M-SC,7.23,248.0,-0.22,1.30,30.55,XY24,NTT,AUTO
43,INFY,0.85,-11.50,51.68,34.24,143113.0,-35967.0,276921.0,1972.0,-30.33,24.0,X-LC,8.70,3.0,-0.25,2.08,1.93,X40,NTT,IT
44,IRCTC,0.66,-30.91,121.35,52.93,226798.0,-83612.0,186896.0,1269.0,-48.87,33.0,H-MC,12.22,119.0,-0.37,1.41,0.66,XY24,BTT,TRAVEL
2,ABBOTINDIA,0.64,-10.43,31.82,18.07,50976.0,-18655.0,160200.0,35195.0,-23.83,46.0,X-MC,11.51,58.0,-0.37,1.21,1.56,X40,ATH,PHARMA
66,SIS,0.59,-29.47,71.93,21.26,56328.0,-32722.0,78310.0,528.0,1840.41,43.0,H-SC,5.89,168.0,-0.58,0.59,7.96,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-5.68,-55.13,224.72,45.70,178277.0,-97482.0,79333.0,26.4,-17.58,53.0,M-SC,18.07,204.0,-0.55,0.60,30.71,XY24,NTT,TRAVEL
55,RAJESHEXPO,-4.04,-73.34,275.36,0.07,100699.0,-100607.0,36570.0,518.0,1240.91,26.0,L-SC,10.12,268.0,-1.00,0.28,0.00,OX40N,NTT,JEWELLERY
69,SURYODAY,-3.79,-27.62,69.99,23.03,90715.0,-49460.0,129611.0,216.0,40.35,47.0,H-SC,5.75,166.0,-0.55,0.98,28.60,XR,NTT,BANKS
32,HATHWAY,-3.64,-42.99,203.97,73.29,126421.0,-46740.0,61980.0,31.4,621.01,33.0,H-SC,20.47,171.0,-0.37,0.47,1.08,XR,NTT,ENTERTAINMENT
76,TRIDENT,-3.40,-33.84,94.25,28.51,57525.0,-31220.0,61034.0,48.0,-14.85,37.0,M-SC,9.00,223.0,-0.54,0.46,4.44,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,-0.86,14.73,81.23,107.91,137591.0,21741.0,169385.0,1641.55,2.99,72.0,M-SC,6.77,220.0,0.16,1.28,27.43,OX40N,ATH,CABLES
66,SIS,0.59,-29.47,71.93,21.26,56328.0,-32722.0,78310.0,528.00,1840.41,43.0,H-SC,5.89,168.0,-0.58,0.59,7.96,OX40N,NTT,MISC
77,TTKPRESTIG,-1.79,-33.27,50.00,0.09,33624.0,-33528.0,67249.0,770.00,56.03,28.0,M-SC,11.96,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
48,KANSAINER,-2.62,-33.86,71.57,13.47,127647.0,-91314.0,178353.0,340.00,-43.11,33.0,H-SC,13.39,162.0,-0.72,1.34,0.26,XY24,NTT,PAINTS
41,INDIGOPNTS,-2.14,-34.88,53.59,0.02,60920.0,-60882.0,113677.0,1408.00,73.55,23.0,M-SC,12.63,222.0,-1.00,0.86,0.00,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-0.79,5.07,12.00,17.67,18970.0,7622.0,158082.0,478.51,26.26,53.0,L-LC,10.87,185.0,0.40,1.19,26.90,XY25,ATH,MINING
27,FINCABLES,-0.86,14.73,81.23,107.91,137591.0,21741.0,169385.0,1641.55,2.99,72.0,M-SC,6.77,220.0,0.16,1.28,27.43,OX40N,ATH,CABLES
1,ABB,-0.30,11.09,31.03,45.57,90186.0,29021.0,290640.0,7934.00,-28.65,64.0,H-MC,8.99,125.0,0.32,2.19,29.06,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.30,11.09,31.03,45.57,90186.0,29021.0,290640.0,7934.00,-28.65,64.0,H-MC,8.99,125.0,0.32,2.19,29.06,AR,NTT,ELECTRICAL
27,FINCABLES,-0.86,14.73,81.23,107.91,137591.0,21741.0,169385.0,1641.55,2.99,72.0,M-SC,6.77,220.0,0.16,1.28,27.43,OX40N,ATH,CABLES
40,INDIAMART,-1.51,-9.76,129.09,106.72,143671.0,-12041.0,111295.0,4810.62,-59.38,35.0,H-SC,8.20,140.0,-0.08,0.84,11.33,AR,ATH,MISC
50,MASFIN,-1.65,-2.33,24.46,21.57,23408.0,-2280.0,95700.0,397.04,-15.99,44.0,H-SC,6.75,167.0,-0.10,0.72,39.66,XR,ATH,FINANCE
85,ZYDUSLIFE,-1.79,-4.55,42.09,35.63,84201.0,-9525.0,200049.0,1286.17,-17.12,49.0,H-MC,4.91,120.0,-0.11,1.51,11.37,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-2.75,-40.40,163.73,57.19,318262.0,-131745.0,194382.0,485.00,-65.95,15.0,X-SC,19.40,81.0,-0.41,1.46,0.00,XY24,NTT,FMCG
12,BATAINDIA,-2.55,-50.40,172.61,35.20,110151.0,-64855.0,63815.0,2096.00,-16.39,24.0,X-SC,22.62,91.0,-0.59,0.48,0.00,X40,NTT,FOOTWEAR
53,PGHH,-0.89,-15.70,56.59,32.01,101862.0,-33520.0,180000.0,17616.87,-42.97,24.0,X-MC,9.67,57.0,-0.33,1.36,0.00,X40,ATH,FMCG
43,INFY,0.85,-11.50,51.68,34.24,143113.0,-35967.0,276921.0,1972.00,-30.33,24.0,X-LC,8.70,3.0,-0.25,2.08,1.93,X40,NTT,IT
9,BAJAJHFL,-2.29,-27.15,113.40,55.47,200995.0,-66042.0,177244.0,181.50,-37.57,26.0,X-MC,17.42,60.0,-0.33,1.33,0.00,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-0.72,0.00,19.34,19.34,38703.0,8.0,200117.0,2252.93,-15.91,46.0,X-MC,1.16,66.0,0.00,1.51,15.94,X40,ATH,INSURANCE
78,UNITDSPR,-0.82,0.60,20.04,20.77,47205.0,1410.0,235554.0,1644.00,-10.79,45.0,X-MC,1.43,64.0,0.03,1.77,7.75,X40N,NTT,BREWERIES
22,DABUR,-1.45,-0.09,43.84,43.70,108651.0,-233.0,247835.0,735.00,-9.31,48.0,X-MC,1.59,70.0,-0.00,1.87,14.94,XY24,BTT,FMCG
59,RELIANCE,-1.38,-11.76,42.00,25.30,74476.0,-23640.0,177323.0,1952.00,-13.88,36.0,X-LC,2.13,21.0,-0.32,1.33,18.78,XY25,BTT,REFINERIES
5,ANGELONE,-0.79,-90.53,1210.89,24.20,350201.0,-276329.0,28921.0,3033.00,-86.38,34.0,X-SC,2.39,97.0,-0.79,0.22,21.12,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-1.90,-34.35,150.03,64.15,234377.0,-81731.0,156220.0,3906.0,-50.29,28.0,X-SC,9.61,85.0,-0.35,1.18,0.0,XY24,BTT,CEMENT
12,BATAINDIA,-2.55,-50.40,172.61,35.20,110151.0,-64855.0,63815.0,2096.0,-16.39,24.0,X-SC,22.62,91.0,-0.59,0.48,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,-2.29,-27.15,113.40,55.47,200995.0,-66042.0,177244.0,181.5,-37.57,26.0,X-MC,17.42,60.0,-0.33,1.33,0.0,X40N,ATH,FINANCE
8,AWL,-2.75,-40.40,163.73,57.19,318262.0,-131745.0,194382.0,485.0,-65.95,15.0,X-SC,19.40,81.0,-0.41,1.46,0.0,XY24,NTT,FMCG
13,BERGEPAINT,-1.83,-20.23,51.87,21.14,94061.0,-46000.0,181339.0,680.0,-22.02,30.0,X-MC,13.96,68.0,-0.49,1.37,0.0,XY24,NTT,PAINTS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-0.79,-90.53,1210.89,24.20,350201.0,-276329.0,28921.0,3033.0,-86.38,34.0,X-SC,2.39,97.0,-0.79,0.22,21.12,X40N,NTT,FINANCE
54,QUESS,-2.15,-35.47,121.36,42.84,50907.0,-23059.0,41947.0,424.0,-57.58,35.0,X-SC,17.17,80.0,-0.45,0.32,0.90,XY24,NTT,MISC
58,RELAXO,-2.86,-56.21,247.77,52.31,157512.0,-81588.0,63572.0,1176.0,-53.34,30.0,X-SC,15.33,92.0,-0.52,0.48,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.55,-50.40,172.61,35.20,110151.0,-64855.0,63815.0,2096.0,-16.39,24.0,X-SC,22.62,91.0,-0.59,0.48,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-0.83,-5.70,31.57,24.08,37794.0,-7234.0,119716.0,1486.0,-6.31,47.0,X-SC,7.96,87.0,-0.19,0.90,9.65,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.59,-17.88,44.99,19.06,125670.0,-60838.0,279328.0,452.00,-55.12,37.0,X-LC,17.17,1.0,-0.48,2.10,1.14,X40,NTT,FMCG
73,TCS,-0.39,-24.41,63.48,23.58,207604.0,-105591.0,327038.0,4311.59,-34.45,31.0,X-LC,9.14,2.0,-0.51,2.46,2.48,X40,ATH,IT
43,INFY,0.85,-11.50,51.68,34.24,143113.0,-35967.0,276921.0,1972.00,-30.33,24.0,X-LC,8.70,3.0,-0.25,2.08,1.93,X40,NTT,IT
75,TMPV,-1.61,-16.33,59.36,33.33,161807.0,-53214.0,272586.0,600.00,-82.53,52.0,X-LC,3.93,4.0,-0.33,2.05,11.44,XY24,NTT,AUTO
34,HCLTECH,-0.72,-10.50,38.36,23.83,83057.0,-25405.0,216519.0,1908.19,-6.12,29.0,X-LC,6.68,8.0,-0.31,1.63,3.88,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.50,-29.72,89.08,32.89,48338.0,-22942.0,54264.0,1800.00,-355.60,61.0,L-MC,12.21,258.0,-0.47,0.41,49.44,XR,NTT,BANKS
67,SONACOMS,-1.86,-9.26,53.28,39.09,48909.0,-9363.0,91796.0,804.02,-28.03,53.0,M-MC,11.09,193.0,-0.19,0.69,29.86,AR,ATH,AUTO
19,COALINDIA,-0.79,5.07,12.00,17.67,18970.0,7622.0,158082.0,478.51,26.26,53.0,L-LC,10.87,185.0,0.40,1.19,26.90,XY25,ATH,MINING
65,SIEMENS,-1.69,-9.70,39.00,25.51,65536.0,-18055.0,168040.0,4671.50,41.07,61.0,H-LC,3.85,51.0,-0.28,1.26,23.75,AR,ATH,ELECTRICAL
27,FINCABLES,-0.86,14.73,81.23,107.91,137591.0,21741.0,169385.0,1641.55,2.99,72.0,M-SC,6.77,220.0,0.16,1.28,27.43,OX40N,ATH,CABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-5.68,-55.13,224.72,45.70,178277.0,-97482.0,79333.0,26.40,-17.58,53.0,M-SC,18.07,204.0,-0.55,0.60,30.71,XY24,NTT,TRAVEL
27,FINCABLES,-0.86,14.73,81.23,107.91,137591.0,21741.0,169385.0,1641.55,2.99,72.0,M-SC,6.77,220.0,0.16,1.28,27.43,OX40N,ATH,CABLES
11,BANDHANBNK,0.24,-12.25,119.29,92.42,291579.0,-34130.0,244429.0,400.00,109.77,72.0,H-SC,8.68,174.0,-0.12,1.84,34.65,XY24,NTT,BANKS
7,ATULAUTO,2.34,-12.49,66.34,45.57,114447.0,-24616.0,172516.0,844.00,3743.24,61.0,M-SC,7.23,248.0,-0.22,1.30,30.55,XY24,NTT,AUTO
83,WHIRLPOOL,-1.47,-27.21,149.92,81.92,125279.0,-31234.0,83564.0,2270.00,-57.38,55.0,M-SC,25.94,210.0,-0.25,0.63,17.38,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.84
1,25,45.92
2,50,78.01


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
MC    30.90
LC    25.32
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.50
X40      24.35
X40N     12.96
AR        9.33
XY25      9.08
XR        8.66
OX40N     7.33
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.27
H-SC    22.67
X-LC    21.82
M-SC    11.00
X-SC     9.47
H-MC     5.11
M-MC     2.11
H-LC     1.26
L-LC     1.19
M-LC     1.05
L-SC     0.70
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.96,-16.46,55.85
IT,11.70,-41.18,111.94
FINANCE,11.17,-40.82,96.23
MISC,6.92,-41.37,99.20
ELECTRICAL,6.56,-7.64,46.92
PAINTS,5.10,-36.40,56.66
INSURANCE,5.01,-2.02,36.94
PHARMA,4.27,-3.87,36.57
AUTO,4.04,-16.24,60.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3475429.0,21
X40,1487992.0,15
X40N,1399997.0,10
XR,1342378.0,12
AR,1333469.0,10
OX40N,821300.0,10
XY25,410191.0,6
SR,306897.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3783662.0,24
X-SC,1546909.0,10
M-SC,1458305.0,14
X-MC,1425096.0,15
X-LC,1373289.0,12
H-MC,401185.0,3
L-SC,198216.0,2
M-LC,153051.0,1
M-MC,105096.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1277796.0      6
           AR         968109.0      5
           XR         860668.0      7
M-SC       XY24       848564.0      6
X-LC       X40        724588.0      6
X-SC       X40N       711830.0      4
           XY24       641340.0      4
X-MC       X40        569665.0      7
           X40N       452347.0      4
H-SC       OX40N      370192.0      4
M-SC       OX40N      350409.0      5
H-SC       SR         306897.0      2
X-LC       XY24       278219.0      2
           X40N       235820.0      2
H-MC       XY24       226798.0      1
X-MC       XY24       202712.0      2
           XY25       200372.0      2
X-SC       X40        193739.0      2
M-SC       XR         182804.0      2
H-MC       AR         174387.0      2
M-LC       XR         153051.0      1
X-LC       XY25       134662.0      2
L-SC       OX40N      100699.0      1
           XR          97517.0      1
M-SC       AR          76528.0      1
H-LC       AR          65536.0      1
M-MC       XY25        56187.0      1
           AR          48909.0      1
L-MC       XR          48338.0      1
L-LC       XY25        18970.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
